In [4]:
from tqdm import tqdm

## Dataset 

In [263]:
# negative = [
#     "Bodo amat  aku gk mau jadi langganan shopee lagi  bye",
#     "min kok saya ga dapat gratis ongkir kalo belanja 150 ribu",
#     "Makin ga enak aja ya mb smua dibatesin",
#     "shopee kenapa harus ganti kebijakan koin shopee si ah kamu kunaon sayang"]

# positive = [
#     "terima kasih untuk photoshoot dari event Ekspress nya semoga Project N Ent semakin lebih baik lagi  mari bekerja lebih keras",
#     "Misal punya saldo di bukalapak 100k tapi barang harga 200k  apa bisa digunakan untuk saldo 100k tersebut dan sisanya dibayarkan melalui metode lain"
#     "lewat kerjasama apik  Van Der Velden dan Spasojevic  Bali united berhasil memperkecil ketertinggalan mereka  bersama Bukalapak",
#     "Esteban Viszcara menggandakan keunggulan Sriwijaya FC atas Bali United   bersama Bukalapak"
# ]

In [264]:
# data = []
# for neg in negative:
#     data.append({
#         "text": neg,
#         "sentiment": "negatif"
#     })
# for pos in positive:
#     data.append({
#         "text": pos,
#         "sentiment": "positif"
#     })

In [265]:
# data

### LOAD DATASET

In [159]:
from textacy import preprocess_text

In [155]:
def get_lexi():
    lexi_positive = open("positive.txt","r")
    lexi_positive_file = lexi_positive.read().split('\n')
    lexi_positive.close()
    
    lexi_negative = open("negative.txt","r")
    lexi_negative_file = lexi_negative.read().split('\n')
    lexi_negative.close()
    
    return lexi_positive_file, lexi_negative_file

lexi_positive_file, lexi_negative_file = get_lexi()

In [165]:
def get_sentiment(text):
    text_clean = preprocess_text(
                    text, 
                    no_punct=True, 
                    lowercase=True, 
                    no_numbers=True, 
                    no_emails=True, 
                    no_phone_numbers=True
                )

    score = 0
    for t in text_clean.split():
        if t in lexi_negative_file:
            score -= 1
        elif t in lexi_positive_file:
            score += 1
        else:
            pass
        
    if score > 0:
        sentiment = "positive"
    elif score < 0:
        sentiment = "negative"
    else:
        sentiment = "neutral"
        
    return text_clean, sentiment

In [228]:
fopen = open("corpus_anotasi.jsonl", "r").read().split("\n")
corpus = [json.loads(f)for f in fopen]

data = []
for i in tqdm(range(len(corpus))):
    c = corpus[i]
    text = c["text"].split('|')[1]
    clean_text, sentiment = get_sentiment(text)
    
    if sentiment != "neutral":
        result = {
            "raw_text": text,
            "text": clean_text,
            "sentiment": sentiment
        }
        data.append(result)

100%|██████████| 10000/10000 [00:08<00:00, 1244.12it/s]


In [229]:
len(data)

4460

In [230]:
print(json.dumps(data[:5], indent=3))

[
   {
      "raw_text": "Tolong tokopedia jika anda memang bertunjuan untuk obyektif lihat bukti yang ada dan minta bukti tidak berdasarkan hanya omongan saja jelas2 itu sepihak dan mengada2 dan dia suda menerima hanya perlu mengirimkan sisanya yang saya terlupa,",
      "text": "tolong tokopedia jika anda memang bertunjuan untuk obyektif lihat bukti yang ada dan minta bukti tidak berdasarkan hanya omongan saja jelas2 itu sepihak dan mengada2 dan dia suda menerima hanya perlu mengirimkan sisanya yang saya terlupa",
      "sentiment": "negative"
   },
   {
      "raw_text": ", Pulsa, makan ,bioskop dllll. Banyak nggak bisa di sebutkan deh. Malah sekarang ovo di pakai tokopedia, mantap dah",
      "text": "pulsa makan bioskop dllll banyak nggak bisa di sebutkan deh malah sekarang ovo di pakai tokopedia mantap dah",
      "sentiment": "positive"
   },
   {
      "raw_text": "min tolong perbaikin pengiriman ya masa masih sejabodetabek lama banget padahal sy liat dirincian udh di pulogeban

## Cleaning

In [231]:
# def set_kata_unik(data):
#     kalimat = []
#     sentimen = []
#     kata_unik = []
#     for kal, sen in data.items():
#         kalimat.append(kal)
#         sentimen.append(sen)
#         for ts in kal.strip().split():
#             kata_unik.append(ts)

#     kata_unik = set(kata_unik)
    
#     return kata_unik

In [232]:
def set_kata_unik(data):
    txt, sen, kata_unik = [], [],[]
    for i in tqdm(range(len(data))):
        d = data[i]
        text = d["text"]
        sentiment = d["sentiment"]

        txt.append(text)
        sen.append(sentiment)

        for ts in text.strip().split():
            ts_clean = ts.lower()
            if ts_clean.isdigit():
                pass
            else:
                if ts_clean not in kata_unik:
                    kata_unik.append(ts_clean)
                    
    return txt, sen, kata_unik

In [233]:
txt, sen, kata_unik = set_kata_unik(data)

100%|██████████| 4460/4460 [00:01<00:00, 2634.66it/s]


In [234]:
# txt, sen, kata_unik

## Vectorisasi

In [235]:
# def set_vector(txt, kata_unik):
#     list_vec = []
#     for tx in txt:
#         kata_unik_ = dict.fromkeys(kata_unik, 0)
#         for t in tx.strip():
#             try:
#                 kata_unik_[t] += 1
#             except Exception as e:
#                 pass
# #                 print(e)
# #                 print(tx)
#         list_vec.append(kata_unik_)
        
    
#     return list_vec

In [236]:
import math

def bag_of_word(txt, kata_unik):
    word_index = []
    for i in tqdm(range(len(txt))):
        tx = txt[i]
        tx_vec = {}
        for ku in kata_unik:
            if ku in tx.lower():
                tx_vec[ku] = 1
            else:
                tx_vec[ku] = 0
        word_index.append(tx_vec)
        
    return word_index

def computeTF(word_index, list_of_text):
    tf = {}
    jumlah_kata = len(list_of_text)
    try:
        for kata, n in word_index.items():
            tf[kata] = n/float(jumlah_kata)
    except:
        pass
        
    return tf

def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    for i in tqdm(range(len(docList))):
        doc = docList[i]
        for word, val in doc.items():
            if val > 0:
                try:
                    idfDict[word] += 1
                except:
                    idfDict[word] = 0
                    idfDict[word] += 1
            else:
                try:
                    idfDict[word] += 0
                except:
                    idfDict[word] = 0
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
        
    return tfidf

In [237]:
list_bag_of_word = bag_of_word(txt, kata_unik)

100%|██████████| 4460/4460 [00:18<00:00, 245.34it/s]


In [238]:
tf = []
for i in tqdm(range(len(list_bag_of_word))):
    tf_i = computeTF(list_bag_of_word[i], txt[i].strip().split())
    tf.append(tf_i)

100%|██████████| 4460/4460 [00:17<00:00, 254.36it/s]


In [239]:
idfs = computeIDF(tf)

100%|██████████| 4460/4460 [00:10<00:00, 410.32it/s]


In [240]:
final_vec = []
for i in tqdm(range(len(tf))):
    tfidf = computeTFIDF(tf[i], idfs)
    value = []
    for k, v in tfidf.items():
        value.append(v)

    value.append(sen[i])
    
    final_vec.append(value)

100%|██████████| 4460/4460 [00:20<00:00, 216.20it/s]


In [241]:
testing = "Terima kasih  Semoga kedepannya bisa lebih baik lagi dalam mengedukasi dan bisa benar2 memastikan apakah penjual patut mendapatkan star seller Selamat hari Selasa"

In [242]:
test = testing.split()

In [243]:
wordDicTest = dict.fromkeys(kata_unik, 0)

In [244]:
for tt in test:
    try:
        wordDicTest[tt] +=1
    except:
        pass

In [245]:
tfTEst = computeTF(wordDicTest, test)

In [246]:
idf_test = computeTFIDF(tfTEst, idfs)

In [247]:
test_vec = []
for k, v in idf_test.items():
    test_vec.append(v)

In [266]:
# test_vec

In [249]:
import math
import operator
k=4

In [250]:
def euclidianDistance(sample1, sample2):
    distance = 0
    for x in range(len(sample1)):
        distance += pow(sample1[x] - sample2[x], 2)

    return math.sqrt(distance)

def getNeighbors(train_set, test_sample, k):
    distances = []
    neighbors = []

    for x in range(len(train_set)):
        dist = euclidianDistance(test_sample, train_set[x])
        distances.append((train_set[x], dist))

    distances.sort(key=operator.itemgetter(1))

    for x in range(k):
        neighbors.append(distances[x][0])
 
    return neighbors

def getResponse(neighbors):
    classVotes = {}

    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
            
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [251]:
# getNeighbors(final_vec, test_vec, k)

In [252]:
# getResponse(getNeighbors(final_vec, test_vec, k))

In [253]:
import json
from sklearn.neighbors import KNeighborsClassifier

In [255]:
X = []
for i in tqdm(range(len(final_vec))):
    vec = final_vec[i]
    X.append(vec[:-1])
    
y = []
for i in tqdm(range(len(sen))):
    s = sen[i]
    if s == "positive":
        y.append(1)
    else:
        y.append(0)

100%|██████████| 4460/4460 [00:00<00:00, 841994.68it/s]


In [256]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [257]:
neigh.predict([test_vec])

array([0])

In [258]:
neigh.predict_proba([test_vec])

array([[0.66666667, 0.33333333]])

In [270]:
true, false = 0,0
for i in tqdm(range(len(X[:100]))):
    x = X[i]
    predict_proba = neigh.predict_proba([x])[0]
    predict = neigh.predict([x])[0]
    
    if predict == y[i]:
        true += 1
    else:
        false += 1
    result = {
        "text": txt[i],
        "predict": "positif" if predict == 1 else "negatif",
#         "predict_proba": {
#             "positif": predict_proba[1],
#             "negatif": predict_proba[0]
#         }
    }
    print(json.dumps(result, indent=3))
    

  2%|▏         | 2/100 [00:00<00:15,  6.24it/s]

{
   "text": "tolong tokopedia jika anda memang bertunjuan untuk obyektif lihat bukti yang ada dan minta bukti tidak berdasarkan hanya omongan saja jelas2 itu sepihak dan mengada2 dan dia suda menerima hanya perlu mengirimkan sisanya yang saya terlupa",
   "predict": "negatif"
}
{
   "text": "pulsa makan bioskop dllll banyak nggak bisa di sebutkan deh malah sekarang ovo di pakai tokopedia mantap dah",
   "predict": "negatif"
}


  4%|▍         | 4/100 [00:00<00:14,  6.53it/s]

{
   "text": "min tolong perbaikin pengiriman ya masa masih sejabodetabek lama banget padahal sy liat dirincian udh di pulogebang dr tigahari yg lalu tp ga sampe jt kurang bagus padahal sy ongkirnya bayar tp ttp sma aja mau gratis bayar pengiriman ga bagus",
   "predict": "negatif"
}
{
   "text": "min lapak saya paket komplit penghilang baret lecet parah dan goresan mobil motor",
   "predict": "negatif"
}


  6%|▌         | 6/100 [00:00<00:14,  6.65it/s]

...

{
   "text": "admin apakah ga salah nih selisih ongkir sampai 50rb sedangkan di sistem barng besar hanya sampai 90rb paling mahal pembeli aja bayar 81rb kenapa minta tambah 50rb lagi",
   "predict": "negatif"
}
{
   "text": "halo min aku pemenang sampai skrg blm di dm juga buat konfirmasi padahal pemenang lain udah dan katanya nnt dikasih kabar lagi via dm ig decicici tp kok blm ada kabar juga",
   "predict": "negatif"
}


 99%|█████████▉| 99/100 [00:15<00:00,  5.90it/s]

{
   "text": "min kenapa ga bisa login jaringan bermasalah terus padahal sinyal ok aplikasi lain lancar jaya",
   "predict": "negatif"
}
{
   "text": "mohon tokopedia klarifikasi seller nakal ngaku sebagai penjual ikan hias ini order sy inv number xviii x number sjak tgl number tidak ada kabar nya",
   "predict": "negatif"
}


100%|██████████| 100/100 [00:15<00:00,  5.98it/s]

{
   "text": "menunggu ampe lebaran monyet kah liat penjual woii cape gue masa iya harus nunggu lama goblok",
   "predict": "negatif"
}


In [262]:
(true/(true+false))*100

81.79372197309416